In [1]:
import numpy as np
import pandas as pd
import json
import os

from disorder.disorder import Disorder
from disorder.entropy import Entropy

In [2]:
path='data/Examples/'
list_of_files=os.listdir(path)

In [3]:
from tqdm import tqdm
from time import time

disordered_comp={}
exc=[]
exc_oxi=[]
exc_large=[]
errors=[]
errors_oxi=[]
errors_large=[]

t1=time()
for i in tqdm(range(len(list_of_files))):
    file=list_of_files[i]
    try:
        ent=Entropy(file=path+file,radius_file='data/all_radii.csv')
        compound={}
        compound['formula']=ent.formula
        compound['ICSD_ID']=ent.material.material.read_id
        compound['group_num']=ent.material.material.space_group
        compound['Z']=ent.z
        if(np.nan not in ent.material.element_radiuses()):
            if(len(ent.material.positions)<501):
                orbits=ent.data
                compound=compound|orbits.to_dict()
                ent=Entropy(file=path+file,radius_file='data/all_radii.csv')
                compound['mixing_entropy']=ent.calculate_entropy('mixing')
                compound['conf_entropy']=ent.calculate_entropy('configurational')
                disordered_comp[file]=compound
            else:
                exc_large.append(file)
                errors_large.append(list(set(ent.material.return_errors())))
        else:
            exc_oxi.append(file)
            errors_oxi.append(list(set(ent.material.return_errors())))
                                    
    except:
        exc.append(file)
        errors.append(list(set(ent.material.return_errors())))
t2=time()
print(t2-t1)

100%|██████████| 26/26 [00:21<00:00,  1.23it/s]

21.075075149536133


In [4]:
with open('disorder_results.json','w') as f:
    json.dump(disordered_comp, f, indent = 4)

In [5]:
err=pd.DataFrame()
err['id']=exc
err['errors']=errors
err.to_csv('excluded_files.csv')

err=pd.DataFrame()
err['id']=exc_large
err['errors']=errors_large
err.to_csv('files_number_of_sites_larger_500.csv')

err=pd.DataFrame()
err['id']=exc_oxi
err['errors']=errors_oxi
err.to_csv('files_oxidation_errors.csv')